# Question Answering with SQUAD

### Setup (if using Google Colaboratory)

In [5]:
! pip install jupyter_nbextensions_configurator
! jupyter nbextensions_configurator enable --user


You should consider upgrading via the '/Users/bekkkkkahhh/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Enabling: jupyter_nbextensions_configurator
- Writing config: /Users/bekkkkkahhh/.jupyter
    - Validating...
      jupyter_nbextensions_configurator 0.4.1 OK
Enabling notebook nbextension nbextensions_configurator/config_menu/main...
Enabling tree nbextension nbextensions_configurator/tree_tab/main...


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# %cd /content/drive/My\ Drive/GaTech/Fall20/NLP-TA/HW5

### Imports

In [1]:
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.notebook import tqdm
random.seed(1)

## Load Data

Read in the pickle files provided to you. We have preprocessed and tokenized all the question answering data form you. Just run the following cells to initialize your word and char to index maps as well as training data. 

For `word2idx` and `char2idx`, index 0 is reserved for *<unk>* and 1 is reserved for *<pad>*. 

In [8]:
! pip install autopep8

     |████████████████████████████████| 121 kB 993 kB/s eta 0:00:01
     |████████████████████████████████| 41 kB 524 kB/s eta 0:00:011
  Created wheel for autopep8: filename=autopep8-1.5.4-py2.py3-none-any.whl size=45287 sha256=2e5b809a641ca7578ad88ae9d23117849846bef627b2961341f882bf292ae611
  Stored in directory: /Users/bekkkkkahhh/Library/Caches/pip/wheels/2c/ad/e2/f5322a230aedd0091b75ec899404e3562d8bb4e7ba0f025cbd
Successfully built autopep8
You should consider upgrading via the '/Users/bekkkkkahhh/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
word2idx_file ='data/word2idx.pickle'
char2idx_file = 'data/char2idx.pickle'
train_pkl = 'data/train-v2.0.json'
valid_pkl = 'data/dev-v2.0.json'

In [3]:
word2idx = pickle.load(open(word2idx_file, 'rb')) # word2idx 
idx2word = {v : k for k, v in word2idx.items()}
word_vocab = list(word2idx.keys())
print("Vocab size:", len(word_vocab))
# for i in range(5):
    # print(word_vocab[i], word2idx[word_vocab[i]], idx2word[word2idx[word_vocab[i]]])
# Print <unk> and <pad>
print(idx2word[0], word2idx['<unk>'])
print(idx2word[1], word2idx['<pad>'])

Vocab size: 23284
<unk> 0
<pad> 1


In [4]:
MAX_WORD_LEN = max([len(x) for x in word2idx.keys()])
MAX_WORD_LEN

30

In [5]:
char2idx = pickle.load(open(char2idx_file, 'rb')) # char2idx 
idx2char = {v : k for k, v in char2idx.items()}
char_vocab = list(char2idx.keys())
print("Unique chars:", len(char_vocab))
print(idx2char[0], char2idx['<unk>'])
print(idx2char[1], char2idx['<pad>'])

Unique chars: 96
<unk> 0
<pad> 1


In [6]:
import util
from util import collate_fn, collate_fn_graph,SQuAD,SQUADwithGraph

ModuleNotFoundError: No module named 'util'

In [7]:
val_df = pd.read_pickle(valid_pkl).reset_index(drop=True)
train_df = pd.read_pickle(train_pkl).reset_index(drop=True)
print("Train samples", len(train_df))
print("Valid samples", len(val_df))
train_df.head()

UnpicklingError: invalid load key, '{'.

In [10]:
sample = train_df.sample(1).iloc[0]
print("Question:", sample.question)
print("Question Ids:", sample.question_ids)
print("Answer:", sample.answer)
print("Answer Ids:", [word2idx[x] for x in sample.answer.split()])
[char_start, char_end] = sample.label
print("Context:", sample.context[char_start - 30 : char_end + 30])
[token_start, token_end] = sample.label_idx
print("Context Ids:", sample.context_ids[token_start - 5: token_end + 5])

NameError: name 'train_df' is not defined

In [8]:
# Counts
print("Train samples", len(train_df))
print("Val samples", len(val_df))

Train samples 8504
Val samples 2013


In [9]:
gold_val_answers = val_df[['id', 'answer']].groupby('id')['answer'].apply(list)
gold_val_answers.head()

id
56ddde6b9a695914005b9628                     [France, France, France, France]
56ddde6b9a695914005b9629    [10th and 11th centuries, in the 10th and 11th...
56ddde6b9a695914005b962a    [Denmark, Iceland and Norway, Denmark, Iceland...
56ddde6b9a695914005b962b                         [Rollo, Rollo, Rollo, Rollo]
56ddde6b9a695914005b962c    [10th century, the first half of the 10th cent...
Name: answer, dtype: object

# 1. BiLSTM + Attention (same as [BiDAF](https://arxiv.org/abs/1611.01603) but different encoder)

> Let's implement a variant of BiDAF using the same LSTM code from HW4 as encoder. 


<img src="img/BiDAF.png">

Here, we will swap out *Contextual Embed Layer*, *Word Embed Layer*, and *Character Embed Layer* with our own LSTM-based encoder. 

In [10]:
# you will need to import again after every change you make
#    autoreload does not work here for some reason (let us 
#    know on piazza if you figure out a fix)
from MyQA import *

In [11]:
sampleInput = train_df.iloc[0]
context = sampleInput.context_ids # tokenized context converted to ids
question = sampleInput.question_ids # tokenized query converted to ids
labels = sampleInput.label_idx # [start_idx, end_idx] of answer
# create char inputs for context and question
context_chars = []
for w in context:
    w = idx2word[w]
    padding = ['<pad>'] * (MAX_WORD_LEN - len(w))
    context_chars += padding + list(w)
context_chars = prepare_sequence(context_chars, char2idx)

question_chars = []
for w in question:
    w = idx2word[w]
    padding = ['<pad>'] * (MAX_WORD_LEN - len(w))
    question_chars += padding + list(w)
question_chars = prepare_sequence(question_chars, char2idx)
context = torch.tensor(context, dtype=torch.long)
question = torch.tensor(question, dtype=torch.long)

In [12]:
# Hyperparameters (you will need to change these)
# Encoding layer
EMBEDDING_DIM = 6
HIDDEN_DIM = 3
LEARNING_RATE = 0.1
LSTM_LAYERS = 1
DROPOUT = 0
EPOCHS = 2
CHAR_EMBEDDING_DIM = 3
CHAR_HIDDEN_DIM = 3
BIDIRECTIONAL_LSTM = True
# Modeling layer
LSTM_LAYERS_MODELING = 2

### Part 1: Encoder

We will use the same POSTagger architecture to encode our passage and questions. The minor distinction here is, you have to encode inputs twice. Once for passage ("context") and one for the question ("query"). *Check that this works before moving on the the next part.*
We will also omit the HighWay Layer for this HW. However, you are welcome to add it for Part 2. 

Implement code in `LSTMEncoder` class in `MyQA.py`. 

In [13]:
directions = 1 if not BIDIRECTIONAL_LSTM else 2
encode = LSTMEncoder(EMBEDDING_DIM, HIDDEN_DIM, 
                     CHAR_EMBEDDING_DIM, CHAR_HIDDEN_DIM, 
                     len(char2idx), len(word2idx), lstm_layers=LSTM_LAYERS, 
                     bidirectional=BIDIRECTIONAL_LSTM, dropout=DROPOUT)
context_enc, query_enc = encode(context, context_chars, question, question_chars)

# verify this with various hyperparameters (by running this cell multiple times)

assert context_enc.shape == (len(context), HIDDEN_DIM*directions)
assert query_enc.shape == (len(question), HIDDEN_DIM*directions)

print("SUCCESS")

SUCCESS


### Part 2: Attention Layer

Implement the Attention Flow Layer same as in BiDAF paper inside `AttentionFlow`. More instructions are included in `MyQA.py`. 



In [14]:
attention = AttentionFlow(HIDDEN_DIM * directions * 3)
G = attention(context_enc, query_enc)
print(G.shape)

# verify this with various hyperparameters (by running this cell multiple times)
assert G.shape == (len(context), HIDDEN_DIM * directions * 4)
print("SUCCESS")


torch.Size([149, 24])
SUCCESS


### Part 3: Modeling Layer

Implement the `ModelingLayer` class in `MyQA.py` same as in BiDAF paper. More instructions are included in `MyQA.py`

In [15]:
modeling = ModelingLayer(HIDDEN_DIM * directions * 4, HIDDEN_DIM, 
                         num_layers=LSTM_LAYERS_MODELING, dropout=DROPOUT, 
                         bidirectional=BIDIRECTIONAL_LSTM)
M = modeling(G)
# verify this with various hyperparameters (by running this cell multiple times)
assert M.shape == (len(context), HIDDEN_DIM * directions)

print("SUCCESS")

SUCCESS


### Part 4: Output Layer

Implement the `OutputLayer` class in `MyQA.py` that will yield start_idx and end_idx vectors.


In [16]:
output = OutputLayer(HIDDEN_DIM*10, HIDDEN_DIM * directions, 
                     HIDDEN_DIM, bidirectional=BIDIRECTIONAL_LSTM)

start, end = output(G, M)

# verify this with various hyperparameters (by running this cell multiple times)
start = start.unsqueeze(0)
end = end.unsqueeze(0)

assert start.shape == (1, len(context))
assert end.shape == (1, len(context))

print("SUCCESS")

SUCCESS


### Finally implement the `BiDAF` class

Now combine all the modules you just wrote into `BiDAF` class in `MyQA.py`

In [17]:
model = BiDAF(EMBEDDING_DIM, HIDDEN_DIM, CHAR_EMBEDDING_DIM, 
               CHAR_HIDDEN_DIM, len(char2idx), len(word2idx), 
               bidirectional=BIDIRECTIONAL_LSTM, phrase_LSTM_layers=LSTM_LAYERS, 
               modeling_LSTM_layers=LSTM_LAYERS_MODELING, dropout=DROPOUT)

start, end = model(context, context_chars, question, question_chars)
start = start.unsqueeze(0)
end = end.unsqueeze(0)

# verify this with various hyperparameters (by running this cell multiple times)
assert start.shape == (1, len(context))
assert end.shape == (1, len(context))

print("SUCCESS")

SUCCESS


# 2. Implement Training and Eval

For this homework, you are required to just implement the train and eval functions. You don't need to train it until convergence or until it reaches high accuracy. However, as you might've noticed, the hyperparameters are the same as HW4. After implementing train and eval, your goal is to train the network for 2 epochs and tune hyperparameters so that the loss is decreasing consistently. We will verify this from your outputs. 

The idea here is to see the network we constructed above work. 

In [63]:
# # Hyperparameters (you will need to change these)
# #Iter: 8503/8504	Avg Train Loss: 9.4279	Avg Val Loss: 9.2280
# # Encoding layer
# EMBEDDING_DIM = 12
# HIDDEN_DIM = 6
# LEARNING_RATE = 0.005
# LSTM_LAYERS = 1
# DROPOUT = 0.1
# EPOCHS = 2
# CHAR_EMBEDDING_DIM = 6
# CHAR_HIDDEN_DIM = 6
# BIDIRECTIONAL_LSTM = True
# # Modeling layer
# LSTM_LAYERS_MODELING = 2

# Hyperparameters (you will need to change these)
# Iter: 8503/8504	Avg Train Loss: 9.2655	Avg Val Loss: 9.0793
# Encoding layer
# EMBEDDING_DIM = 32
# HIDDEN_DIM = 16
# LEARNING_RATE = 0.01
# LSTM_LAYERS = 2
# DROPOUT = 0.1
# EPOCHS = 2
# CHAR_EMBEDDING_DIM = 16
# CHAR_HIDDEN_DIM = 16
# BIDIRECTIONAL_LSTM = True
# # Modeling layer
# LSTM_LAYERS_MODELING = 2

#Epoch1: Iter: 8503/8504	Avg Train Loss: 9.3010	Avg Val Loss: 9.0261
#Epoch2: Iter: 8503/8504	Avg Train Loss: 8.8123	Avg Val Loss: 8.7616
# Encoding layer
# EMBEDDING_DIM = 64
# HIDDEN_DIM = 32
# LEARNING_RATE = 0.01
# LSTM_LAYERS = 2
# DROPOUT = 0.1
# EPOCHS = 2
# CHAR_EMBEDDING_DIM = 32
# CHAR_HIDDEN_DIM = 32
# BIDIRECTIONAL_LSTM = True
# # Modeling layer
# LSTM_LAYERS_MODELING = 2


# Encoding layer
EMBEDDING_DIM = 128
HIDDEN_DIM = 64
LEARNING_RATE = 0.01
LSTM_LAYERS = 2
DROPOUT = 0.1
EPOCHS = 2
CHAR_EMBEDDING_DIM = 64
CHAR_HIDDEN_DIM = 64
BIDIRECTIONAL_LSTM = True
# Modeling layer
LSTM_LAYERS_MODELING = 2

In [64]:
def get_eval_scores(predictions):
    """2D list of [[ID, PRED], [ID, PRED], ...]"""
    f1 = exact_match = 0
    for row in predictions:
        [ID, PRED] = row
        goldans = gold_val_answers.loc[ID]
        f1 += max(compute_f1(g, PRED) for g in goldans)
        exact_match += max(compute_exact(g, PRED) for g in goldans)
    num_samples = len(predictions)
    return f1 / num_samples, exact_match / num_samples

In [66]:
def train_BiDAF(epoch, model, loss_function, optimizer):
    train_loss = 0
    train_examples = len(train_df)
    itr = 0
    for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
        # for i, row in train_df.iterrows():
        context = row.context_ids # tokenized context converted to ids
        question = row.question_ids # tokenized query converted to ids
        labels = row.label_idx # [start_idx, end_idx] of answer
        #############################################################################
        # TODO: Implement the training loop
        # Find the gradient with respect to the loss and update the model 
        #   parameters using the optimizer.
        # `context` and `question` are both already tokenized and converted to ids
        # You need to prepare your character level input. Feel free to use the 
        #   prepare_sequence method for this. 
        #############################################################################
        
        # create char inputs for context and question           
        context_chars = []
        for w in context:
            w = idx2word[w]
            padding = ['<pad>'] * (MAX_WORD_LEN - len(w))
            context_chars += padding + list(w)
        context_chars = prepare_sequence(context_chars, char2idx)

        question_chars = []
        for w in question:
            w = idx2word[w]
            padding = ['<pad>'] * (MAX_WORD_LEN - len(w))
            question_chars += padding + list(w)
        question_chars = prepare_sequence(question_chars, char2idx)
        
        # use the model
        context = torch.tensor(context, dtype=torch.long)
        print('context', context.shape)
        question = torch.tensor(question, dtype=torch.long)
        start, end = model(context, context_chars, question, question_chars)
        
        # prepare data
        start = start.reshape(1,-1)
        end = end.reshape(1,-1)
        labels = torch.tensor(labels)
        se = torch.cat((start, end), dim=0)
        
        # caculate the loss here
#         loss = 2 * loss_function(se, labels)
        loss = loss_function(se, labels)
        
        # update the loss
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        # update train_loss
        train_loss += loss.item()
        
#         print(torch.norm(model.outputLayer.fc_GM.weight))
               
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        if ((i+1) % 100) == 0:
            avg_train_loss = train_loss / i
#             print("\nLoss:", avg_train_loss)
            avg_val_loss, f1_avg, exact_match_avg = eval_BiDAF(model, loss_function, rand_samples=10)
            print("Iter: {}/{}\tAvg Train Loss: {:.4f}\tAvg Val Loss: {:.4f}\tVal F1: {:.0f} \tVal ExactMatch: {:.0f}".format(i, train_examples, avg_train_loss, avg_val_loss, f1_avg, exact_match_avg))
    
    avg_train_loss = train_loss / train_examples
    avg_val_loss, f1_avg, exact_match_avg = eval_BiDAF(model, loss_function, rand_samples=1000)        
    print("Iter: {}/{}\tAvg Train Loss: {:.4f}\tAvg Val Loss: {:.4f}\tVal F1: {:.0f} \tVal ExactMatch: {:.0f}".format(i, train_examples, avg_train_loss, avg_val_loss, f1_avg, exact_match_avg))
    return model

def eval_BiDAF(model, loss_function, rand_samples=-1):
    val_loss = 0
    if rand_samples != -1:
        validation_set = val_df.sample(rand_samples)
    else:
        validation_set = val_df
    val_examples = len(validation_set)
    predictions = []
    with torch.no_grad():
        for i, row in tqdm(validation_set.iterrows(), total=len(validation_set)):
            # for i, row in validation_set.iterrows(): # if tqdm is not installed 
            context = row.context_ids # tokenized context converted to ids
            orig_context = prepare_sequence(context, idx2word)
            question = row.question_ids # tokenized query converted to ids
            [true_start, true_end] = row.label_idx # [start_idx, end_idx] of answer
            pred_start, pred_end = None, None # maxidx of the softmax outputs
            #############################################################################
            # TODO: Implement the training loop
            # Find the gradient with respect to the loss and update the model 
            #   parameters using the optimizer.
            # `context` and `question` are both already tokenized and converted to ids
            # You need to prepare your character level input. Feel free to use the 
            #   prepare_sequence method for this. 
            #############################################################################
            
            # create char inputs for context and question           
            context_chars = []
            for w in context:
                w = idx2word[w]
                padding = ['<pad>'] * (MAX_WORD_LEN - len(w))
                context_chars += padding + list(w)
            context_chars = prepare_sequence(context_chars, char2idx)

            question_chars = []
            for w in question:
                w = idx2word[w]
                padding = ['<pad>'] * (MAX_WORD_LEN - len(w))
                question_chars += padding + list(w)
            question_chars = prepare_sequence(question_chars, char2idx)

            # use the model
            context = torch.tensor(context, dtype=torch.long)
            question = torch.tensor(question, dtype=torch.long)
            start, end = model(context, context_chars, question, question_chars)
            
            # prepare data
            start = start.reshape(1,-1)
            end = end.reshape(1,-1)
            labels = torch.tensor([true_start, true_end])
            se = torch.cat((start, end), dim=0)

            # caculate the loss here
            loss = loss_function(se, labels)
            
            #update loss
            val_loss += loss.item()
            
            # get the prediction here.
            pred_start, pred_end = int(start.argmax(dim = 1)[0]), int(end.argmax(dim = 1)[0])
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            predictions.append([row.id, orig_context[pred_start : pred_end + 1]])
            
    f1_avg, exact_match_avg = get_eval_scores(predictions)
    avg_val_loss = val_loss / val_examples
    return avg_val_loss, f1_avg*100, exact_match_avg*100

#############################################################################
# TODO: define loss function and optimizer to be used in train_BiDAF 
#############################################################################
model = BiDAF(EMBEDDING_DIM, HIDDEN_DIM, CHAR_EMBEDDING_DIM, 
               CHAR_HIDDEN_DIM, len(char2idx), len(word2idx), 
               bidirectional=BIDIRECTIONAL_LSTM, phrase_LSTM_layers=LSTM_LAYERS, 
               modeling_LSTM_layers=LSTM_LAYERS_MODELING, dropout=DROPOUT)
device = torch.device("cpu")
model.to(device)

from torch.autograd import Variable

def loss_fn(data, labels):
    loss = Variable(torch.zeros(1))
    for d, label in zip(data, labels):
        loss -= torch.log(d[label]).cpu()
    return loss

# Define loss function
loss_function = loss_fn

# Define optimizer
parameters = filter(lambda p: p.requires_grad, model.parameters())
        
    # in the paper: We use the AdaDelta (Zeiler, 2012) optimizer
    # with a minibatch size of 60 and an initial learning rate of 0.5, for 12 epochs.

optimizer = optim.Adadelta(parameters, lr=LEARNING_RATE)
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
for epoch in range(1, EPOCHS + 1):
    model = train_BiDAF(epoch, model, loss_function, optimizer)
#     eval_BiDAF(model, loss_function, optimizer)

#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################

context torch.Size([149])
context torch.Size([149])
context torch.Size([149])
context torch.Size([149])
context torch.Size([149])
context torch.Size([149])
context torch.Size([149])
context torch.Size([149])
context torch.Size([149])
context torch.Size([149])



KeyboardInterrupt: 

In [61]:
EMBEDDING_DIM

64

In [67]:
 len(char2idx), len(word2idx)

(96, 23284)